In [3]:
import sys
sys.path.append("../api")
import pandas as pd
from functions import bertify, fit_score_model

In [4]:
df = pd.read_csv('../data/02/emotions.csv')

In [5]:
df.is_positive.value_counts()

0    28369
1    17711
Name: is_positive, dtype: int64

In [11]:
y = df['feeling']
X = bertify(df['lemma'])

You try to use a model that was created with version 1.2.0, however, your version is 1.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Encoded !


In [12]:
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression(max_iter=1500, multi_class="multinomial", C=100)
# model = fit_score_model(model, X, y)

0.47974879402930737
              precision    recall  f1-score   support

       anger       0.53      0.43      0.47       622
        fear       0.58      0.50      0.54       551
         fun       0.20      0.05      0.08       356
       happy       0.54      0.64      0.58      2513
        hate       0.31      0.13      0.19       239
        love       0.51      0.33      0.40      1160
     neutral       0.41      0.49      0.45      1646
     sadness       0.51      0.55      0.53      2255
       worry       0.38      0.40      0.39      1645

    accuracy                           0.48     10987
   macro avg       0.44      0.39      0.40     10987
weighted avg       0.47      0.48      0.47     10987

In [13]:
# import joblib
# path = f'../models/cuml_LogisticRegression.joblib'
# joblib.dump(model, path, compress=3)

In [14]:
# from sklearn.svm import LinearSVC
# model_2 = LinearSVC(max_iter=1500)
# model_2 = fit_score_model(model_2, X, y)

0.482388277054701
              precision    recall  f1-score   support

       anger       0.54      0.34      0.42       622
        fear       0.58      0.47      0.52       551
         fun       0.50      0.01      0.01       356
       happy       0.52      0.68      0.59      2513
        hate       0.35      0.03      0.05       239
        love       0.50      0.29      0.37      1160
     neutral       0.42      0.51      0.46      1646
     sadness       0.50      0.58      0.54      2255
       worry       0.39      0.38      0.39      1645

    accuracy                           0.48     10987
   macro avg       0.48      0.36      0.37     10987
weighted avg       0.48      0.48      0.46     10987

In [27]:

from xgboost import XGBClassifier, callback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

xgb = XGBClassifier(
    use_label_encoder=False,
    learning_rate=0.01, 
    n_estimators=1500,
    predictor="gpu_predictor",
    tree_method="gpu_hist",
    gpu_id=0,
    objective='multi:softprob',
    min_child_weight=1.5,
    subsample=0.8,
    sampling_method="gradient_based",
    max_depth=20,
    gamma=1.5,
    colsample_bytree=0.8 
)

#Extracting different classes
class_to_idx = {x:i for i,x in enumerate(sorted(df.sentiment.unique()))}
idx_to_class = {i:x for i,x in enumerate(sorted(df.sentiment.unique()))}
print(class_to_idx)
print(idx_to_class)
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, train_size=0.85,
                                                    random_state=42)

early_stopping_rounds = 20
callbacks = [callback.EarlyStopping(rounds=early_stopping_rounds,
                                        save_best=True)]

{'anger': 0, 'enthusiasm': 1, 'fear': 2, 'happy': 3, 'hate': 4, 'love': 5, 'neutral': 6, 'relief': 7, 'sadness': 8, 'surprise': 9, 'worry': 10}
{0: 'anger', 1: 'enthusiasm', 2: 'fear', 3: 'happy', 4: 'hate', 5: 'love', 6: 'neutral', 7: 'relief', 8: 'sadness', 9: 'surprise', 10: 'worry'}


In [32]:
xgb.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)], eval_metric=["merror"], callbacks=callbacks, verbose=True)

[0]	validation_0-merror:0.16381	validation_1-merror:0.57413


KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot 
results = xgb.evals_result()
epochs = len(results['validation_0']['merror'])
x_axis = range(0, epochs)


fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['merror'], label='Train')
ax.plot(x_axis, results['validation_1']['merror'], label='Test')
ax.legend()
pyplot.ylabel('merror')
pyplot.title('XGBoost merror')
pyplot.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = xgb.predict(X_test)
preds = bst.predict(dtest, ntree_limit=0, training=False)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [148]:
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import RandomizedSearchCV

# folds = 4
# param_comb = 40

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }

# random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=1, cv=skf.split(X,y), verbose=3, random_state=1001 )

In [149]:
# random_search.fit(X, y)

Fitting 4 folds for each of 40 candidates, totalling 160 fits
[CV 1/4] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0; total time=  42.0s
[CV 2/4] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0; total time=  42.3s
[CV 3/4] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0; total time=  42.4s
[CV 4/4] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0; total time=  42.5s
[CV 1/4] END colsample_bytree=0.8, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.6; total time= 1.4min
[CV 2/4] END colsample_bytree=0.8, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.6; total time= 1.4min
[CV 3/4] END colsample_bytree=0.8, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.6; total time= 1.4min
[CV 4/4] END colsample_bytree=0.8, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.6; total time= 1.4min
[CV 1/4] END colsample_bytree=0.8, gamma=1

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7f6990441dd0>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           eval_metric='merror', gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.02,
                                           max_delta_step=None, max_depth=None,
                                           min_chil...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                  

In [150]:
# print('\n All results:')
# print(random_search.cv_results_)
# print('\n Best estimator:')
# print(random_search.best_estimator_)
# print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
# print(random_search.best_score_ * 2 - 1)
# print('\n Best hyperparameters:')
# print(random_search.best_params_)
# results = pd.DataFrame(random_search.cv_results_)
# results
# #results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([42.11209416, 85.00206012, 85.93747759, 65.8242327 , 54.27026933,
       45.03359067, 64.50765902, 33.45770437, 65.46005875, 44.22959161,
       49.7930373 , 36.73288029, 63.83013451, 86.4965322 , 86.56124043,
       48.97886789, 55.03147089, 54.20877081, 47.92296022, 73.69206941,
       55.02010632, 31.24818277, 76.31324762, 31.76625061, 30.43675667,
       73.27762645, 76.18129599, 63.62187028, 54.93193001, 31.19178474,
       49.78756261, 76.09491116, 75.86240059, 83.79472476, 84.85177368,
       86.40079623, 31.76039732, 48.9852407 , 49.74325514, 86.04982704]), 'std_fit_time': array([0.16432383, 0.23746821, 0.07997075, 2.57179164, 0.07448725,
       0.10300881, 0.17346926, 0.01956169, 0.09782388, 0.05300846,
       0.06408824, 0.08993655, 0.12225519, 0.09889082, 0.13478865,
       0.10473419, 0.08173004, 0.09653312, 0.06880713, 0.04655265,
       0.09731736, 0.02466617, 0.09324099, 0.05588482, 0.03715853,
       0.11329309, 0.14220961, 0.07096

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_min_child_weight,param_max_depth,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,42.112094,0.164324,0.204120,0.002349,1.0,5,3,5,1.0,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.393141,0.404937,0.399621,0.400379,0.399519,0.004207,40
1,85.002060,0.237468,0.332277,0.002272,0.6,1,5,1.5,0.8,"{'subsample': 0.6, 'min_child_weight': 1, 'max...",0.424525,0.435156,0.435011,0.430163,0.431214,0.004353,1
2,85.937478,0.079971,0.333072,0.002427,0.8,5,5,1,0.8,"{'subsample': 0.8, 'min_child_weight': 5, 'max...",0.423287,0.434646,0.433991,0.427687,0.429903,0.004688,7
3,65.824233,2.571792,0.345176,0.002609,1.0,5,5,5,0.6,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.422122,0.432025,0.432244,0.425648,0.428010,0.004310,10
4,54.270269,0.074487,0.250445,0.001539,0.8,1,4,1,1.0,"{'subsample': 0.8, 'min_child_weight': 1, 'max...",0.410180,0.418845,0.420738,0.415453,0.416304,0.004011,24
5,45.033591,0.103009,0.252532,0.003553,1.0,10,4,1.5,0.6,"{'subsample': 1.0, 'min_child_weight': 10, 'ma...",0.409306,0.418117,0.420083,0.415526,0.415758,0.004061,27
6,64.507659,0.173469,0.343913,0.003199,1.0,1,5,5,0.6,"{'subsample': 1.0, 'min_child_weight': 1, 'max...",0.421831,0.431151,0.429768,0.426740,0.427373,0.003575,11
7,33.457704,0.019562,0.200311,0.005941,0.8,1,3,2,0.8,"{'subsample': 0.8, 'min_child_weight': 1, 'max...",0.396199,0.403845,0.401660,0.399141,0.400211,0.002852,36
8,65.460059,0.097824,0.327452,0.006695,0.8,1,5,0.5,0.6,"{'subsample': 0.8, 'min_child_weight': 1, 'max...",0.423797,0.431734,0.434938,0.430236,0.430176,0.004056,6
9,44.229592,0.053008,0.251030,0.001390,0.8,1,4,1.5,0.6,"{'subsample': 0.8, 'min_child_weight': 1, 'max...",0.408796,0.418263,0.421539,0.416254,0.416213,0.004679,25


In [ ]:
# from sklearn.metrics import classification_report, accuracy_score
# y_pred = xgb.predict(X_test)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))